# Spectral graphing 

In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interact, widgets, interactive
from signal_processing import baseline_als

import plot_utils

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
df = pd.read_csv('merged_data_with_labels.csv', index_col=0, low_memory=False)

In [4]:
# select a x-axis basis for our plot
w = list(np.arange(450, 2500).astype(str))
basis = np.array(w).astype(float)

In [5]:
# Select the REE collection
df1 = df[df['dataset'] == 'OF8619']  # <--- change this to 'USGSsplib07a' or 'OF7923'

The following plot shows the raw and corrected spectrum for each sample selected. The spectra are shown as corrected using two methods. Shown in the same color as the curve class itself, we show the standard hull-correction. In purple is the Asymmetric Least Squares (ALS) fit. The values of Lambda and P can be adjusted to control the fitting. 

Other background removal methods are probably available, but we want to choose something that can be easily applied across any sample without any fine tuning or parameterization.

In [6]:
df1.head()

,cm_ree_labels,ID,Mineral_Name,Mineral_Group,Formula,dataset,350,351,352,353,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,2,2155a,Ancylite,Ancylite Group,"(Ce,La)Sr(CO3)2(OH) · H2O",OF8619,0.178708,0.176169,0.174322,0.171489,...,0.287711,0.286648,0.285574,0.284534,0.283488,0.282632,0.281855,0.281146,0.280420,0.279790
1,2,2156,Ancylite,Ancylite Group,"(Ce,La)Sr(CO3)2(OH) · H2O",OF8619,0.345439,0.344923,0.344561,0.341901,...,0.440031,0.438991,0.438029,0.437181,0.436327,0.435411,0.434513,0.433637,0.432825,0.431991
2,2,2155b,Ancylite,Ancylite Group,"(Ce,La)Sr(CO3)2(OH) · H2O",OF8619,0.162516,0.160701,0.159454,0.157421,...,0.361510,0.360665,0.359890,0.359077,0.358304,0.357598,0.356949,0.356368,0.355754,0.355215
3,2,2157,Ancylite-?,Ancylite Group,"(Ce,La)Sr(CO3)2(OH) · H2O",OF8619,0.131739,0.131160,0.130220,0.127734,...,0.114268,0.113663,0.113129,0.112580,0.112111,0.111643,0.111204,0.110811,0.110418,0.110173
4,2,2370,Thalenite-(Y),NaN,Y3Si3O10F,OF8619,0.120644,0.119710,0.118695,0.117396,...,0.109176,0.108603,0.108044,0.107537,0.107070,0.106586,0.106208,0.105826,0.105548,0.105195


In [7]:
# Make an interactive plot

# MAKE SURE YOU UNCHECK Save Figure Button before you move to the next plot 
# (this almost certainly can be improved)

def spectral_plot(idx_slider, save_button):
    save = save_button
    lam, p = 1e7, 0.99
    fig, ax = plot_utils.plot(df1.iloc[idx_slider], dark_flag=0.5, flag_color='gray')
    y = df1.iloc[idx_slider][w].astype(float)
    ax.plot(basis, baseline_als(y, lam=lam, p=p), linestyle='dashed', 
            c='purple', label='als fit')
    als_corr = y / baseline_als(y, lam=lam, p=p)
    ax.plot(basis, als_corr/ np.amax(als_corr), c='purple',
            label='als-correction')
    ax.legend()
    if save:
        cm_ree_label = df1.iloc[idx_slider]['cm_ree_labels']
        id_name = df1.iloc[idx_slider]['ID']
        min_name = df1.iloc[idx_slider]['Mineral_Name']
        save_name = f'{cm_ree_label}_{id_name}_{min_name}.PNG'
        fig.savefig(save_name, facecolor='white')
    save_button = False
    return fig

    
save_button = widgets.ToggleButton(description='Save Figure', value=False)

#out = w.Output(layout=widgets.Layout(border = '1px solid black'))

def on_button_clicked(b):
    print("Button clicked.")

#output = widgets.Output()
#display(save_button, output)
        
idx_slider = widgets.IntSlider(min=0, max=df1.shape[0]-1, step=1, value=0, continuous_update = False)

interactive_plot = interactive(spectral_plot, 
                               idx_slider = idx_slider, save_button=save_button)

output = interactive_plot.children[-1]
#output.layout.height = '700px'
interactive_plot


interactive(children=(IntSlider(value=0, continuous_update=False, description='idx_slider', max=487), ToggleBu…